In [ ]:
import polars as pl
from huggingface_hub import snapshot_download
from datasets import load_dataset
import os
import sys
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
sys.path.append("..")
%load_ext autoreload
%autoreload 2


In [ ]:
import os

from synthetic_data.tasks.writing import _get_gutenberg_subset

gutenberg_pq = _get_gutenberg_subset(2)

In [ ]:
gutenberg_pq.shape

In [ ]:
from IPython.display import Markdown
import re

from synthetic_data.utils import ldictl
from synthetic_data.tasks.writing import GutenbergBacktranslation
first_sample = gutenberg_pq.sample(n=4)
task = GutenbergBacktranslation()

input = first_sample.to_dicts()
print(input)
task.format_input_conversation(ldictl(input))